In [1]:
#importing libraries, modules
import numpy as np
import tensorflow as tf
import os

In [2]:
################################################################################
# Class for loading data
################################################################################
from spektral.data import Dataset, Graph
class MyDataset(Dataset):

    def __init__(self, n_samples, feats, path, **kwargs):
        self.n_samples = n_samples
        self.feats = feats
        self.path = path
        super().__init__(**kwargs)

    def read(self):
        def make_graph(i):
            feature=np.loadtxt(self.path+'GNN_inputs_RVE_'+str(i+1)+'.csv',delimiter=',')
            num_grains=int(len(feature)/6)
            feature=feature.reshape((num_grains,6)) #reshape the feature matrix
            
            neigbor=np.loadtxt(self.path+'Neighbor_matrix_RVE_'+str(i+1)+'.csv',delimiter=',')
            neigbor=neigbor.reshape((num_grains,num_grains))
     
            stress=np.loadtxt(self.path+'stress_output.csv',delimiter=',')
            y=np.array(stress[i])
            
            return Graph(x=feature, a=neigbor, y=y)

        # We must return a list of Graph objects
        return [make_graph(i) for i in range(self.n_samples)]

In [3]:
################################################################################
# Load dataset
################################################################################
from spektral.transforms.normalize_adj import NormalizeAdj

num_RVEs=2000
num_features=5
#directory of input files
path=''
dataset = MyDataset(num_RVEs,num_features, path, transforms=NormalizeAdj())

In [4]:
from spektral.data import DisjointLoader, BatchLoader
# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split
idxs = np.random.permutation(len(dataset))
split = int(0.75 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]

batch_size = 32
loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=5000)
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=100)

In [5]:
################################################################################
# Build model
################################################################################
from spektral.layers import GCSConv, GlobalAvgPool, GlobalSumPool, GraphMasking
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
class Net(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCSConv(64, activation="relu")
        self.conv2 = GCSConv(128, activation="relu")
        #self.conv3 = GCSConv(256, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, i = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        #x = self.conv3([x, a])
        output = self.global_pool([x,i])
        output = self.dense(output)

        return output

In [6]:
################################################################################
# Compile model
################################################################################
model = Net()
learning_rate = 1e-3
optimizer = Adam(learning_rate)
model.compile(optimizer=optimizer, loss="mse")
model.summary()

In [ ]:
################################################################################
# Fit model
################################################################################
model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=100)

In [ ]:
################################################################################
# Evaluate model
################################################################################
from tensorflow.keras.losses import MeanSquaredError
loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)
print("Testing model")
loss = 0
Pred=np.array([])
Targ=np.array([])
for batch in loader_te:
    inputs, target = batch
    predictions = model(inputs, training=False)
    Pred=np.append(Pred,predictions)
    Targ=np.append(Targ,target)
loss /= loader_te.steps_per_epoch
print("Done. Test loss: {}".format(loss))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(9,9))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.xlabel('Actual stress (MPa)', fontsize=20)
plt.ylabel('Predicted stress (MPa)', fontsize=20)
plt.scatter(Targ,Pred)
plt.plot(Targ,Targ,'r')
plt.savefig('GNN_uni.jpg')

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
print(mean_absolute_error(Targ,Pred))
print(np.sqrt(mean_squared_error(Targ,Pred)))
print(explained_variance_score(Targ,Pred))